# Twisted Probabilities, Uncerntainty, and Prices
by [Lars Peter Hansen](https://larspeterhansen.org/), [Bálint Szőke](https://www.balintszoke.com/), Lloyd S. Han and [Thomas J.Sargent](http://www.tomsargent.com/), you could find the latest draft [here](https://larspeterhansen.org/research/papers/). The notebook presents the code, table and figures accompanying section 6 in the paper

We illustrate effects of concerns about robustness in three environments using a model of capital accumulation with adjustment costs proposed by [Eberly & Wang (2011)](https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf). We modify their model to expose investment returns  to long-run risks and make investors concerned  about misspecifications of them. Three distinct example economies environments feature:
* a single capital stock
* two capital stocks with a common exposure to long-run risk
*  two capital stocks with only one being exposed to long-run risk

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline-model" data-toc-modified-id="Baseline-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline model</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Calibration" data-toc-modified-id="Calibration-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calibration</a></span></li></ul></li><li><span><a href="#Single-capital-stock-case" data-toc-modified-id="Single-capital-stock-case-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Single capital stock case</a></span><ul class="toc-item"><li><span><a href="#Parameter-alternatives" data-toc-modified-id="Parameter-alternatives-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parameter alternatives</a></span></li><li><span><a href="#Planner's-Problem" data-toc-modified-id="Planner's-Problem-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Planner's Problem</a></span></li></ul></li><li><span><a href="#Two-capital-stocks" data-toc-modified-id="Two-capital-stocks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Two capital stocks</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Planner's-problem" data-toc-modified-id="Planner's-problem-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Planner's problem</a></span></li><li><span><a href="#Symmetric-returns-to-two-capital-stocks" data-toc-modified-id="Symmetric-returns-to-two-capital-stocks-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Symmetric returns to two capital stocks</a></span></li><li><span><a href="#Asymmetric-returns-to-two-capital-stocks" data-toc-modified-id="Asymmetric-returns-to-two-capital-stocks-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Asymmetric returns to two capital stocks</a></span></li></ul></li></ul></div>

In [3]:
# Import Necessary packages
using Optim
using Roots
using NPZ

include("newsets_utils.jl")

shock_price (generic function with 1 method)

## Baseline model

### Setup

Aggregate output is proportional to a single capital stock with a constant productivity parameter $\mathcal{A}$. A representative household cares about consumption $C$ with instantaneous utility $\delta\log C$. Under the baseline model, investment $I$ affects capital $K$ according to

\begin{align}
dK_t &= \left[\frac{I_t}{K_t} - \frac{\phi}{2}\left(\frac{I_t}{K_t}\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right)\right]K_t dt + (.01)\sigma_k K_t\cdot dW_t \label{eqn:model}\\
dZ_t &= \left(\widehat\alpha_z -\widehat \kappa Z_t \right)dt + \sigma_z \cdot dW_t \nonumber
\tag{1}
\end{align}

with adjustment cost parameter $\phi$. With zero investment, the rate of change of capital is $(.01)\left(\widehat\alpha_k + \widehat\beta Z_t\right)$, where $Z$ is an exogenously specified continuous-time scalar autoregressive process  that puts long-term risks into  returns on capital. Under baseline model $(1)$, the mean of $Z$ is  ${\overline z} = {\frac {{\widehat \alpha}_z} {{\widehat \kappa}}}$.

he aggregate resource constraint is $C + I = {\cal A} K$. It is convenient to pose a robust planner's problem in terms of an investment-capital ratio $D_t \doteq\frac{I_t}{K_t}$ and $\log K_t$ that satisfy
\begin{align*}
d \log K_t = \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}{2}} \right] dt + (.01)\sigma_k \cdot dW_t.
\end{align*}
Denote state variable as $X_t = [\log K_t,\ Z_t]'$ and instantaneous utility function
\begin{align*}
\upsilon(X_t, D_t) = \delta\log\left({\mathcal A}- D_t\right) + \delta\log K_t.
\end{align*}



### Calibration

The basis of our quantitative analysis is an empirical model of  aggregate consumption dynamics. Our code for VAR estimation could be found in __VAR Estimation__ section of Git hub repository [Tenuous Belief](https://github.com/lphansen/TenuousBeliefs).

We follow [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) by fitting a trivariate VAR for macroeconomic time series that contain information about long-term consumption growth, namely,
log consumption growth, the difference between logs of business income and consumption, and the difference between logs of personal dividend income and
consumption.

The time series are quarterly data from 1948 Q1 to 2018 Q2. Our consumption measure is per capita consumption of non-durables and services from NIPA Table 1.1.5. Business income is measured as proprietor's income plus corporate profits per capita and the series are from NIPA Table 1.12. Personal dividend income is from NIPA Table 2.1. By including proprietors' income in addition to corporate profits, we use a broader measure of business income than [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf), who used only corporate profits. Moreover, [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) did not include  personal dividends in their VAR analysis.
We restrict  all three time series to have a common martingale component by imposing a known cointegration relation among them.

We convert the discrete time VAR estimates to baseline parameters $({\widehat \alpha}_k,  {\widehat \beta}, {\widehat \alpha}_z, {\widehat \kappa} )$ and $(\sigma_c, \sigma_z)$ by setting ${\widehat \alpha}_z=0$ and $\widehat{\beta}=1$ and matching the dynamics of the VAR implied long-term consumption growth forecast with those of $Z$.
As a result, we obtain the following parameters for the baseline model of consumption $(1)$:

\begin{align} 
& \begin{matrix}
{\widehat \alpha}_c  & = & .484 & & {\widehat \beta} &= &1 \cr
{\widehat \alpha}_z  &= & 0 &  & {\widehat \kappa} & = & .014 \end{matrix} \cr
& \sigma = \begin{bmatrix}
(\sigma_c)' \cr (\sigma_z)' \end{bmatrix}  =  \begin{bmatrix} .477 & 0 \cr  .011 & .025 \end{bmatrix} .
\label{estimations}
\end{align}

We set the household's subjective discount rate equal to $\delta=.002.$

In more detail, we choose $\widehat{\alpha}_c$ and $(1-\widehat{\kappa})$ to match the VAR implied unconditional mean of consumption growth and autoregressive coefficient of expected long-term consumption growth, respectively. In addition, we set $(\sigma_c, \sigma_z)$ such that $(1, \widehat{\kappa}^{-1})\sigma\sigma'(1, \widehat{\kappa}^{-1})'$ is equal to the VAR implied covariance matrix of the one-step-ahead forecast error for consumption growth and expected long-term consumption growth. We achieve identification by imposing a lower triangular structure with positive diagonal elements on $\sigma$.

In [1]:
#==============================================================================#
#                           Setting PARAMETERS
#==============================================================================#
delta = 0.002;

# (1) Baseline model
alpha_z_hat = 0.0;
kappa_hat = 0.014;

alpha_c_hat = 0.484;       # consumption intercept (estimated) 
beta_hat = 1.0;
sigma_c = [0.477, 0.0  ];   # consumption exposure (= exposure of single capital)
sigma_z = [0.011, 0.025];

## Single capital stock case

### Parameter alternatives

A planner is concerned about  alternative models for  capital evolution than baseline model $(1)$ in the following parametric class

\begin{align}
d \log K_t &= \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left( \alpha_k +  \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}2 } \right] dt + (.01)\sigma_k \cdot dW^H_t \label{eqn:model2} \\
dZ_t &= \left(\alpha_z - \kappa Z_t \right)dt + \sigma_z \cdot dW^H_t \nonumber
\tag{2}
\end{align}

where $W^H$ denotes a distorted shock process of the form:

\begin{equation}
dW_t = H_t dt + d{W}_t^H,
\end{equation}

Uncertainty about $(\alpha_k,\beta)$ describes unknown components in  returns to investment while uncertainty about $(\alpha_z,\kappa)$ captures an unknown growth evolution process. We represent a model of the form $(2)$ by restricting the drift distortion $H$ for $W$ to satisfy

\begin{equation}\label{eq:affine_drift}
H_t = \eta(X_t) \doteq {\eta }_0  + {\eta }_1 Z_t
\end{equation}

and using $(1)$  and  $(2)$ to deduce the following restrictions on $\eta_0$ and $\eta_1$ as functions of $(\alpha_k, \beta, \alpha_z,  \kappa)$:
\begin{align}
\sigma {\eta}_0  = \begin{bmatrix} {\alpha}_k - {\widehat \alpha}_k \cr {\alpha}_z  -{\widehat \alpha}_z \end{bmatrix}, \hspace{2cm}
\sigma {\eta}_1  = \begin{bmatrix} \beta - {\widehat \beta}  \cr {\widehat \kappa} - { \kappa}  \end{bmatrix}
\tag{3}
\end{align}
where
\begin{align}
\sigma \doteq \begin{bmatrix} (\sigma_k)' \cr (\sigma_z)' \end{bmatrix} .
\end{align}

Pairs $(\eta_0, \eta_1)$ that satisfy restrictions $(3)$ represent models having parametric form $(2)$.

We restrict the set of alternative models by using a quadratic twisting function $\xi(z)$. Following the method of section 3.2.1 we ensure that the set $\mathcal{M}(\xi)$ includes a prespecified quartet $({\widetilde \alpha}_k, {\widetilde \beta}, {\widetilde \alpha}_z, {\widetilde \kappa})$ by solving an instance of $(3)$ for ${\widetilde \eta}_0$ and ${\widetilde \eta}_1$, then setting

\begin{align}
\xi(x) =  \left\vert {\widetilde \eta}_0 + {\widetilde \eta}_1 z \right\vert^2.
\tag{4}
\end{align}

For the quantitative examples below,  either we twist the pair $\left({\widetilde \alpha}_z, {\widetilde \kappa}\right)$ and form $\xi^{[\kappa]}$, or we twist the pair $({\widetilde \alpha}_z, {\widetilde \beta})$ and form $\xi^{[\beta]}$.


 To set worrisome parameters $\left({\widetilde \alpha}_z, {\widetilde \kappa}\right)$ or $({\widetilde \alpha}_z, {\widetilde \beta})$, we target the entropy of the implied worst-case model $M^{H^*}$ relative to the baseline model $(1)$. As discussed in section 3, relative entropy $\Delta^*\left(M^H\right)$ of martingale process $M^H$ equals one half the expectation of $|H_t|^2$ under the $H$ induced probability model. As a result, we can parameterize relative entropy of the worst-case model using the scalar ${\sf q}$:
 \begin{align}
 \Delta^*\left(M^{H^*}\right) = {\frac {{\sf q}^2}  2}
\end{align}

One can think of ${\sf q}$ as a measure of the magnitude of a probability distortion. We calibrate worrisome models to induce specific $\mathsf{q}$ values.

To see how state-dependence of a quadratic twisting function influences this calibration, We decomopose relative entropy into state-independent and state-dependent parts $\xi(Z_t)=\xi_0 + \widetilde{\xi}(Z_t)$ and suppose that twisted relative entropy constraint is satisfied with equality to arrive at

\begin{align}
\frac{\mathsf{q}^2}{2} = \Delta^*\left(M^{H^*}\right) = \frac{\xi_0}{2} + \lim_{\delta \downarrow 0} \frac{\delta}{2}\int \exp(-\delta \tau)E\left(M^{H^*}_{\tau} \widetilde{\xi}(Z_{\tau}) \mid   \mathcal F_0\right)d\tau \doteq \xi_0 + \Omega(\xi) .
\end{align}

The last equality defines $\Omega(\xi)$ as the part of relative entropy that arises from state-dependence in $\xi$. 

### Planner's Problem

To capture a desire for robustness to parametric misspecifications represented by the class $(2)$, we suppose that the twisting function has the form $(4)$ and seek a function $V(z, k; \ell) = \log k + \nu(z; \ell)$ that satisfies the robust planner's HJB equation
\begin{align}\label{hjb:robust}
0 = & \max_{d} \min_{h} \ \ - \delta \nu(z; \ell) + \delta \log ({\mathcal A} - d) + \left[d - {\frac {\phi} 2} d^2 +  (.01) \left({\widehat \alpha}_k + {\widehat \beta} z - \frac{(.01)|\sigma_k^2|}{2}\right)\right] \cr
& \quad\quad + \frac{\partial \nu(z; \ell)}{ \partial z}\left( {\widehat \alpha}_z -{\widehat \kappa} z \right) + \frac{1}{2}\frac{ \partial^2 \nu(z; \ell)}{\partial z^2} |\sigma_z|^2 \nonumber \\
& \quad\quad + (.01) \left[\sigma_k \cdot h\right] + \frac{\partial \nu(z; \ell)}{\partial z}\left[\sigma_z \cdot h\right]  +  \frac{\ell}{2}\left[ |h|^2 - \xi(z) \right].
\tag{5}
\end{align}

The last line of $(5)$ depicts how concerns about robustness enter the HJB equation via $h$. The minimizing choice of $h$ depends on the state $z$ through the marginal value function:

\begin{align}
H^*(z; \ell) & \doteq \begin{bmatrix}h^*_k \\ h^*_z \end{bmatrix} = -\frac{1}{\ell}\left[ (.01) \sigma_k + \frac{\partial  \nu(z ; \ell)}{\partial z}\sigma_z  \right]. \tag{6}
\end{align}

The  maximizing choice of $d$ solves:

\begin{align}\label{eqn:d_star}
 1 - \phi d^* =   {\frac \delta  {{\mathcal A} - d^*}}
\end{align}

where $d^*$ denotes the chosen investment-capital ratio. Notice that  $d^*$ is constant and does not depend on the underlying state variables, so the optimal consumption-capital ratio $c^*=\mathcal{A}-d^*$ is also constant. Because neither $c^*$ nor $d^*$ is affected by
robustness concerns,  under the baseline model capital and  consumption dynamics are not altered by robustness concerns. We write the implied dynamics for the logarithm of consumption as:

\begin{align}
d \log C_t  = (.01)\left({\widehat \alpha}_c  +  {\widehat \beta} Z_t\right) dt + (.01)\sigma_c \cdot dW_t \label{eq:baseline_consumption}
\end{align}

where

\begin{align}\label{eq:alpha_c}
  {\widehat \alpha_c} = 100\left(d^* - \frac{\phi}{2}\left(d^*\right)^2\right) +  {\widehat \alpha}_k  - {\frac {(.01) |\sigma_k|^2}2 } \hspace{15mm}\text{and}\hspace{15mm} \sigma_c = \sigma_k.
\end{align}

In Appendix C.1, we show that the value function $\nu(z; \ell)$ is quadratic in $z$. As a consequence, $h^*$ is affine in $z$. The multiplier $\ell^*$ is then determined by solving

\begin{align*}
\ell^*(z_0) = \arg\max_{\ell>0} \ \nu(z_0; \ell)
\end{align*}

where $z_0$ is an initial value of $Z$. We can find the worst-case model by substituting $\ell^*(z_0)$ in equation $(6)$.

## Two capital stocks

### Setup
We now extend investment opportunities by adding a second productive capital that can be used to produce the common consumption good with constant productivity. Under the baseline model, investment $I^{(1)}$ and $I^{(2)}$ affect two capital stocks $K^{(1)}$ and $K^{(2)}$ according to
\begin{align*}
dK_t^{(1)} &= K_t^{(1)} \left[\left({\frac {I_t^{(1)}}{K_t^{(1)}}}  - {\frac {\phi_1}  2}  \left(\frac{I_t^{(1)}}{K_t^{(1)}} \right)^2 + (.01) \left( {\widehat \alpha}_1 + {\widehat \beta}_1 Z_t   \right)\right) dt + (.01) \sigma_1   \cdot dW_t \right] \\
dK_t^{(2)} &=  K_t^{(2)} \left[ \left({\frac{I_t^{(2)}}{K_t^{(2)}}}  -   {\frac {\phi_2}  2} \left(\frac{I_t^{(2)}}{K_t^{(2)}} \right)^2 + (.01) \left( {\widehat \alpha}_2 + {\widehat \beta}_2 Z_t \right) \right)  dt + (.01)  \sigma_{2}  \cdot dW_t \right] \\
dZ_t &= \left({\widehat \alpha}_z   - {\widehat \kappa} Z_t \right)dt + \sigma_z \cdot dW_t
\end{align*}
subject to the aggregate resource constraint

\begin{align*}
C_t + I_t^{(1)} + I_t^{(2)} = \mathcal{A}_1  K_t^{(1)} + \mathcal{A}_2 K_t^{(2)}.
\end{align*}

The two sectors are  identical in their technology parameters $\left(\mathcal{A}_1, \phi_1, \widehat{\alpha}_1\right)=\left(\mathcal{A}_2, \phi_2, \widehat{\alpha}_2\right)$ and exposures to the Brownian shocks, $\sigma_1=\sigma_2$, but  can differ in  exposures to long-run risk, so that $\widehat{\beta}_1 \neq \widehat{\beta}_2$. To study how multiple capital stocks and  heterogeneity in their exposures to  long-run risk  affect decision rules and the worst-case model, we consider two cases:

* __ex post heterogeneity:__ two capital stocks possess identical evolution equations but are exposed to idiosyncratic shocks that give rise to a non degenerate distribution of capital ex post. This case features a trade-off between diversification and adjustment costs similar to studied by [Eberly & Wang (2011)](https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf). We study how concerns about model misspecification affect this trade-off.

* __ex ante heterogeneity:__ two capital stocks differ in their evolution equations, the first capital stock is immune to long-run risk because $\widehat{\beta}_1=0$, while the second capital stock is exposed to it because  $\widehat{\beta_2}>0$. %and alter the technology parameters so that the induced average consumption dynamics is identical to the single-capital economy.

Like the case with a single capital stock, it is convenient to use the investment ratios $D_t^{(1)} \doteq {\frac {I_t^{(1)}}{K_t^{(1)}}}$ and $ D_t^{(2)} \doteq {\frac {I_t^{(2)}}{K_t^{(2)}}}$ as controls, and the logarithm of aggregate capital, $\log K_t \doteq \log\left(K^{(1)}+K^{(2)}\right)$, and the long-run risk component, $Z$, as state variables. However, with  costly reallocation between the two capital stocks, the distribution of capital becomes an additional endogenous state variable that we express with the ratio

\begin{align*}
R_t \doteq \frac{K^{(2)}_t}{K^{(1)}_t+ K^{(2)}_t} = \frac{K^{(2)}_t}{K_t} \in [0, 1].
\end{align*}

Adjustment costs prevent the household from setting  $R_t$ ideally at every instant; instead the household  influences its motion at each instant  by setting the investment ratios $D_t^{(1)}$ and $D_t^{(2)}$.

The state vector is then $X_t \doteq \left[\log K_t, \ R_t, \ Z_t\right]'$. 

### Planner's problem

In Appendix C, we show that the value function of the robust planner is additively separable in $\log K$, making the optimal investment ratios $d_1^*$ and $d_2^*$ become functions of $(R, Z)$ only. Implied equilibrium consumption is
\begin{align}
\log C_t = \log K_t + \log\Big([\mathcal{A}_1 - d^*_1(R_t, Z_t)](1-R_t) + [\mathcal{A}_2-d^*_2(R_t, Z_t)]R_t \Big).
\end{align}
The worst-case model ceases to share the  parametric form of  the baseline model, but the distortion $h^*$ remains Markov and depends only on the contemporaneous capital distribution, $R_t$, and the long-run risk state $Z_t$.

To induce independent variation in the two capital return processes, we presume that there are three underlying Brownian motions with volatility vectors
\begin{align*}
\sigma_1 &= \mathsf{s}\begin{bmatrix}
.477  \\
0 \\
0
\end{bmatrix},\quad \sigma_2 = \mathsf{s}\begin{bmatrix}
0 \\
.477 \\
0
\end{bmatrix}, \quad \sigma_z = \begin{bmatrix}
.011\sqrt{.5} \\
.011 \sqrt{.5}\\
.025
\end{bmatrix} ,
\end{align*}
where $\mathsf{s}$ is a scaling parameter that we introduce to control  effects of diversification on aggregate consumption volatility. Multiplying the first two entries of $\sigma_z$ by $\sqrt{.5}$ ensures that, despite having an extra shock, the local volatility of the long-run risk state, $|\sigma_z|$, is unchanged relative to the single capital economy.

The state vector is
\begin{align*}
X_t \doteq \left[\log K_t, \ L_t, \ Z_t-\bar{z}\right]' \quad\quad \log K_t \doteq \log\left(K^{(1)}_t+ K^{(2)}_t\right)\quad\quad L_t \doteq \log K^{(2)}_t - \log K^{(1)}_t
\end{align*}

For numerical stability concerns, we used $L_t$ defined above instead of capital distribution ratio $R_t$. One can get $R_t$ from $L_t$ using the one-to-one mapping $R_t=\exp(L_t)/(1+\exp(L_t))$. 

The period utility function is
\begin{align*}
\upsilon(X, D) &= \delta \log\left((1-R)\left(\mathcal{A}_1 - D^{(1)}\right) + R\left(\mathcal{A}_2 - D^{(2)}\right) \right) + \delta \log K
\end{align*}

Denote  expected capital growth $E_t\left[dK^{(i)}_t/K^{(i)}_t\right]$ for $i=1, 2$ as
$$\varphi_i\left(D_t^{(i)}, Z_t\right) \doteq D_t^{(i)} - \frac{\phi_i}{2}\left(D_t^{(i)}\right)^2 + (.01)\left(\widehat\alpha_z + \widehat\beta Z_t\right)$$
State variables then follow
\begin{align*}
d\log K_t &= \left[\varphi_1 (1-R_t) + \varphi_2 R_t - \frac{(.01)^2\left|\sigma_1(1-R_t) + \sigma_2 R_t\right|^2}{2}\right]dt + %\\
%& \hspace{1cm} +
(.01)\left[\sigma_1(1-R_t) + \sigma_2 R_t\right]\cdot dW_t \\
dL_t &= \left[\varphi_2 - \varphi_1 -  \frac{(.01)^2}{2}\left(|\sigma_2|^2 - |\sigma_1|^2 \right)\right] dt + (.01)\left[\sigma_2 -\sigma_1\right]\cdot dW_t \\
dZ_t &= - \widehat\kappa \left(Z_t - \bar z\right)dt + \sigma_z \cdot dW_t
\end{align*}
Using Ito's lemma, we can derive the following  dynamics for $R_t$:
\begin{align*}
dR_t &= R_t(1-R_t)\left[\varphi_2 - \varphi_1 +
(.01)^2\left(|\sigma_1|^2(1-R_t) - |\sigma_2|^2 R_t + \sigma'_1\sigma_2(2R_t-1)\right)\right] dt + \\
& \hspace{8cm} + R_t(1-R_t)(.01)\left[\sigma_2 -\sigma_1\right]\cdot dW_t .
\end{align*}
Let $\sigma$ denote the stacked volatility matrix
\begin{align*}
\sigma(X_t) \doteq \begin{bmatrix}
(.01)\left(\sigma'_1(1-R_t) + \sigma'_2 R_t\right) \\
(.01)[\sigma_2-\sigma_1]' \\
\sigma'_z
\end{bmatrix} .
\end{align*}
\noindent We seek a value function  $V(X) = \log K + \nu(L, Z)$ that solves the HJB equation
\begin{align}
0 &= \max_{d^{(1)}, d^{(2)}}\min_{h} \ \delta\log\left((1-r)\left(\mathcal A_1 - d^{(1)}\right) + r\left(\mathcal A_2 - d^{(2)}\right) \right) - \delta\nu(l, z) + \frac{\ell}{2}\left[|h|^2 - \xi(z)\right] \nonumber \\
& \quad\quad + \left[\varphi_1(1-r) + \varphi_2r - \frac{(.01)^2\left[\sigma_1(1-r) + \sigma_2 r\right]^2}{2} + (.01)[(1-r)\sigma_1 + r\sigma_2]\cdot h\right]  \nonumber \\
& \quad\quad + \nu_l(l, z)\left[ \varphi_2 - \varphi_1 - \frac{(.01)^2}{2}\left(|\sigma_2|^2  - |\sigma_1|^2 \right) + (.01)[\sigma_2 - \sigma_1]\cdot h\right] \nonumber \\
& \quad\quad + \nu_z(l, z)\left[ -\widehat{\kappa}(z-\bar z) + \sigma_z \cdot h\right] + \frac{1}{2}\text{tr}\left(V_{xx}\sigma\sigma'\right)\label{eq:HJB}
\end{align}
where
\begin{align*}
\text{tr}\left(V_{xx}\sigma\sigma'\right) &= (.01)^2|\sigma_2-\sigma_1|^2 \nu_{ll}(l, z) +  2 (.01)\left([\sigma_2 - \sigma_1]\cdot \sigma_z\right)\nu_{lz}(l, z) +  |\sigma_z|^2 \nu_{zz}(l, z).
\end{align*}

\noindent We assume that a  Bellman-Isaacs condition holds so that first-order conditions can be stacked
\begin{align}
\frac{\delta(1-r)}{(1-r)(\mathcal A_1 - d^{(1)}(l, z))+r(\mathcal A_2 - d^{(2)}(l, z))} &= \left(1-\phi_1d^{(1)}(l, z)\right)\left[1 - r - \nu_l(l, z)\right] \label{eq:d1_opt} \\
\frac{\delta r}{(1-r)(\mathcal A_1 - d^{(1)}(l, z))+r(\mathcal A_2 - d^{(2)}(l, z))} &= \left(1-\phi_2d^{(2)}(l, z)\right)\left[r + \nu_l(l, z)\right] \label{eq:d2_opt} \\
h(l, z, \ell^*) &= - \frac{1}{\ell^*} \sigma'(r)\begin{bmatrix}
1 \\
\nu_l(l, z) \\
\nu_z(l, z)
\end{bmatrix} . \label{eq:h}
\end{align}

These equations determine  optimal investment-capital ratios $d^{(1)}(l, z)$ and $d^{(2)}(l, z)$, and also the worst-case drift distortion $h(l, z)$.
Here  $\ell^*$ is the multiplier that makes the minimizing agent's constraint bind for a given initial $(l_0, z_0)$:

\begin{align*}
\ell^*(l_0, z_0) = \text{arg}\max_\ell \ \ \nu(l_0, z_0, \ell).
\end{align*}

### Symmetric returns to two capital stocks

Consider the case with  __ex post__ heterogeneity only, that is, set ${\widehat \beta}_1 = {\widehat \beta}_2 = .5$ and $({\mathcal A}_1, \phi_1, \widehat\alpha_1) = ({\mathcal A}_2, \phi_2, \widehat\alpha_2)=({\mathcal A}, \phi, \widehat\alpha_k)$ using values from calibration results in section 1.2 . Because diversification can reduce aggregate consumption volatility, we increase the exposures $\sigma_1=\sigma_2$ relative to the single-capital economy by setting $\mathsf{s}=1.32$, so that average volatility of equilibrium consumption growth  equal  that estimated from U.S. data.


### Asymmetric returns to two capital stocks

We now turn to \emph{ex ante} heterogeneity by studying an economy in which the two capital stocks are exposed to long run uncertainty asymmetrically.
 In particular, we set $\widehat{\beta}_1=0$ and $\widehat{\beta}_2=.5$ and choose $\mathsf{s}=1.14$ so that unconditional means of the drift and volatility terms of equilibrium consumption growth are equal to those estimated from U.S. data.
